In [4]:
import pandas as pd
import numpy as np
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import re
from bs4 import BeautifulSoup

driver = webdriver.Chrome()

# Access capabilities
capabilities = driver.capabilities

driver.get("https://www.melon.com/chart/age/index.htm?chartType=YE&chartGenre=KPOP&chartDate=2021")
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

def melon_collector(url):
    time.sleep(5)
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 제목 가져오기
    title=driver.find_elements(by=By.CLASS_NAME, value='ellipsis.rank01') # 제목

    title2=[]
    for i in title:
        title2.append(i.text)

    del title2[30:]    
    
    # 가수 가져오기
    singer=driver.find_elements(by=By.CLASS_NAME, value='ellipsis.rank02') # 가수

    singer2=[]
    for i in singer:
        singer2.append(i.text)

    del singer2[30:]
    
    # 가사 가져오기
    
    song_info = soup.find_all('div', attrs={'class': 'ellipsis rank01'})
    
    # Top 30만 추출
    songid = []

    for i in range(30):
        try:
            songid.append(re.sub(r'[^0-9]', '', song_info[i].find("a")["href"][43:])) # 곡 id정보 추출
        except:
            songid.append('')
            continue # 1992년 먼지가 되어와 같이 멜론에서 현재 들을 수 없는 노래 제외
        
    lyrics=[]

    for i in songid:
        try:
            driver.get("https://www.melon.com/song/detail.htm?songId=" + i)
            time.sleep(2)
            lyric=driver.find_element(by=By.CLASS_NAME, value="lyric")
            lyrics.append(lyric.text)  
        except: # 가사가 없는경우
            lyrics.append('')
            continue
    lyrics2 = []

    for i in lyrics:
        lyrics2.append(i.replace("\n"," "))
        
    df=pd.DataFrame({"제목":title2,"가수":singer2,"가사":lyrics2})
    
    # 저장하기
    df.to_csv(f'멜론{start}.csv', index = False)
# 크롤링한 데이터를 csv형태로 저장하였고, 이 때 현재 멜론에서 들을 수 없는 노래나 가사가 등록되어있지 않은 노래들은 제외하고 가져와줍니다. 

start = 1990
url = 'https://www.melon.com/chart/age/index.htm?chartType=YE&chartGenre=KPOP&chartDate='

while start<=2021:
    new_url = url + str(start)
    melon_collector(new_url)
    print(start,'완료') # 완료 여부 표시
    start += 1


1990 완료
